## Fitting profiles with Markov Chain Monte Carlo
This is a jupyter notebook intended to fit stellar density profiles to 
a bunch of clusters simulated. It fits the exponential radius, ellipticity, angular position, background density and center shift in ra and dec.

The code runs reading the position of the stars in a catalog and fitting the pars. After the fitting, the results should be compared to the
truth table (the parameters of the clusters simulated).

Open a terminal and run the following commands to create and activate a new environment and install dependencies:
```
conda create -p $HOME/.conda/envs/emcee_env
conda activate emcee_env
conda install -c astropy emcee
conda install -c conda-forge matplotlib
conda install astropy
conda install -c astropy corner
conda install ipykernel
pip install astropy
ipython kernel install --user --name=emcee_env
```
Activate the environment (if you are running the code in Linea's jupyter notebook). Restart the kernel on the jupyter
notebook and load the new environment (clicking on the env on the top right selection).</br>
Ok, now you are ready to run the code. </br>
You can run this code on any machine since you follow the steps above to create an env and install dependencies.

TODO (Ana Clara, Amanda):
- insert documentation in the functions;
- plot the chains to make sure the cuts in steps are ok to all pars;
- format the results into a single file;
- add cells to compare the results to the truth table (output of fits).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import astropy.io.fits as fits
import corner
from multiprocessing import Pool


Below are a few functions (that will be documented in near future!) to provide the results of the fitting.
This is very similar to the main example available on https://emcee.readthedocs.io/en/stable/tutorials/line/ .
Please, visit that page to have a big picture of the fitting.

In [ ]:
def lnprior(theta):
    """Calculates if pars are within limits.

    Parameters
    ----------
    theta : list
        Set of parameters.

    Returns
    -------
    float
        Zero or -infinity
    """
    re, ell, th, bg, xc, yc = theta
    if 0.01 < re < 20. and 0.0 < ell < 0.6 and -1.8 < th < 1.8 and 0. < bg < 500. and -3. < xc < 3. and -3. < yc < 3.:
        return 0
    return -np.inf


def lnlike(theta, RA, DEC):
    """Calculates likelihood of a specific set of parameters.

    Parameters
    ----------
    theta : list
        Set of parameters.
    RA : list
        RA of stars, in degrees.
    DEC : list
        DEC of stars, in degrees.

    Returns
    -------
    float
        2*ln likelihood
    """

    global R0, R1

    re, ell, th, bg, xc, yc = theta
    dX = (RA-(ra0+xc/60.))*np.cos(np.radians(dec0))
    dY = (DEC-(dec0+yc/60.))
    ri = 60.*np.sqrt(((dX*np.cos(np.radians(100*th))-dY*np.sin(np.radians(100*th)))/(1.-ell))
                     ** 2.+(dX*np.sin(np.radians(100*th))+dY*np.cos(np.radians(100*th)))**2.)
    s0 = (len(RA)-bg*3600.)/((1-ell)*2.*np.pi*re**2)

    #R0 = 0.5
    #R1 = 0.25
    idbg = (ri < R0) & (ri > R1)
    area_annulus_arcmin = 3600. * np.pi * (R0 ** 2 - R1 ** 2)
    bg = float(len(RA[idbg]))/(area_annulus_arcmin)  # stars/arcmin^2
    N_star = (len(RA[R < R0])-(robg * 3600. * np.pi * R0 ** 2))
    s0 = (len(RA)-bg*3600.)/((1-ell)*2.*np.pi*re**2)

    if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
        return -np.inf
    return 2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))


def lnprob(theta, RA, DEC):
    """Accounts two previous functions.

    Parameters
    ----------
    theta : list
        set of parameters.
    RA : list
        RA of stars in degrees.
    DEC : list
        DEC of stars in degrees.

    Returns
    -------
    float
        Likelihood. If within the limits it is a number.
    """
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, RA, DEC)


# Some definitions:
R0 = 0.5
R1 = 0.25
# ra0, dec0, reinf, resup, thmin, thmax = 67.3156, -23.8368, 0.1, 10., 0.0, 180.
HPX_, ra0_, dec0_, re_kick_, ell_kick_, pa_kick_, dist = np.loadtxt(
    'star_clusters_simulated.dat', usecols=(0, 6, 7, 8, 9, 10, 12), unpack=True)

# Converting exponential radius from parsecs to arcmin
re_kick = 60. * (180. / np.pi) * np.arctan(re_kick_ / dist)

# Reading the catalog of stars
hdu = fits.open('des_mockcat_for_detection.fits', memmap=True)
RA_ = hdu[1].data.field('ra')
DEC_ = hdu[1].data.field('dec')
#MAGG = hdu[1].data.field('mag_g_with_err')
#MAGGERR = hdu[1].data.field('magerr_g')
#MAGR = hdu[1].data.field('mag_g_with_err')
#MAGRERR = hdu[1].data.field('magerr_r')
hdu.close()

fff = open('results.dat', 'w')

print('# HPX re(0,+,-) ell(0,+,-) pa(0,+,-) robg(0,+,-) shift_ra(0,+,-) shift dec(0,+,-)', file=fff)
for i in range(len(ra0_)):
    # print('Pars (ideal):', HPX_[i], re_kick[i], ell_kick_[i], pa_kick_[i], '----', '0.00', '0.00')
    reinf = 0.5 * re_kick[i]
    resup = 1.5 * re_kick[i]
    thmin = pa_kick_[i] - 45.
    thmax = pa_kick_[i] + 45.
    ra0 = ra0_[i]
    dec0 = dec0_[i]

    cutout_size_RA = R0 / np.cos(np.deg2rad(dec0))

    cond = (RA_ > ra0 - cutout_size_RA) & (RA_ < ra0 +
                                           cutout_size_RA) & (DEC_ > dec0 - R0) & (DEC_ < dec0 + R0)
    RA = RA_[cond]
    DEC = DEC_[cond]

    R = 60.*np.sqrt(((np.cos(np.radians(dec0))*(ra0-RA))**2.)+(DEC-dec0)**2)

    re_ml = (reinf + resup)/2.
    ell_ml = 0.1
    th_ml = (thmin + thmax)/200.
    idbg = (R < R0) & (R > R1)
    area_annulus_arcmin = 3600. * np.pi*(R0**2 - R1**2)
    robg = float(len(RA[idbg]))/(area_annulus_arcmin)  # stars/arcmin^2
    N_star = (len(RA[R < R0])-(robg*3600.*np.pi * R0 ** 2))
    s_ml = N_star/(2.*np.pi*re_ml**2)
    xc_ml = 0.0
    yc_ml = 0.0
    result = re_ml, ell_ml, th_ml, robg, xc_ml, yc_ml

    ndim, nwalkers = 6, 300  # 300
    pos = [result + 1e-4*np.random.randn(ndim) for j in range(nwalkers)]
    with Pool() as pool:
        sampler = emcee.EnsembleSampler(
            nwalkers, ndim, lnprob, args=(RA, DEC), pool=pool)
        # sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(RA, DEC))
        sampler.run_mcmc(pos, 300)  # 200
    samples = sampler.chain[:, 100:, :].reshape((-1, ndim))

    samples[:, 2] = 100.*(samples[:, 2])
    re_mcmc, ell_mcmc, th_mcmc, robg_mcmc, xc_mcmc, yc_mcmc = map(lambda v: (
        v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84], axis=0)))
    print(int(HPX_[i]), re_mcmc[0], re_mcmc[1], re_mcmc[2], ell_mcmc[0], ell_mcmc[1], ell_mcmc[2],
          th_mcmc[0], th_mcmc[1], th_mcmc[2], robg_mcmc[0], robg_mcmc[1], robg_mcmc[2],
          xc_mcmc[0], xc_mcmc[1], xc_mcmc[2], yc_mcmc[0], yc_mcmc[1], yc_mcmc[2], file=fff)
    
    # Plotting best-fitting pars
    fig = corner.corner(samples, labels=["$re$", "$ell$", "$th$", r"$\rho_{bg}$", r"$\Delta\alpha$", r"$\Delta\delta$"], truths=[
                        re_mcmc[0], ell_mcmc[0], th_mcmc[0], robg_mcmc[0], xc_mcmc[0], yc_mcmc[0]], quantiles=[0.16, 0.5, 0.84], show_titles=True, plot_contours=True)
    plt.savefig(str(int(HPX_[i])) + '_plus.png')
    plt.close()
    '''
    L = np.zeros(len(RA))
    dX = (RA-(ra0+xc_mcmc[0]/60.))*np.cos(np.radians(dec0))
    dY = (DEC-(dec0+yc_mcmc[0]/60.))
    ri = 60.*np.sqrt(((dX*np.cos(np.radians(100*th_mcmc[0]))-dY*np.sin(np.radians(100*th_mcmc[0])))/(
        1.-ell_mcmc[0]))**2.+(dX*np.sin(np.radians(100*th_mcmc[0]))+dY*np.cos(np.radians(100*th_mcmc[0])))**2.)
    L = (np.exp(-ri/re_mcmc[0]))
    '''
    # Saving chains
    fig1, axes = plt.subplots(6, figsize=(10, 7), sharex=True)
    samples = sampler.get_chain()
    labels = ["rexp", "ell", "theta", "rho_bg", "Ddelta", "Dalpha"]
    for j in range(ndim):
        ax = axes[j]
        ax.plot(samples[:, :, j], "k", alpha=0.3)
        ax.set_xlim(0, len(samples))
        ax.set_ylabel(labels[j])
        ax.yaxis.set_label_coords(-0.1, 0.5)
    axes[-1].set_xlabel("step number")
    plt.savefig(str(int(HPX_[i])) + '_chains.png')
    plt.close()
fff.close()
#a = open('J0429_stars_list_profile.dat','w')
# for j in range(len(RA)):
#    if (L[j]<0.):
#        L[j]=0.
#    print >> a, RA[j], DEC[j], MAGG[j], MAGR[j], ERRG[j], ERRR[j], SM[j], L[j]
# a.close
